In [1]:
import scanpy as sc
import jax
from ot_pert.metrics import compute_metrics, compute_metrics_fast, compute_mean_metrics
import functools
import numpy as np

In [2]:
adata_preds_from_atac = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_atac_with_predictions_peakVI.h5ad")
adata_preds_from_rna = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_with_predictions_peakVI.h5ad")
adata_target_atac = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_test_processed_peakvi.h5ad")
adata_atac_train = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_train_processed_peakvi.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [3]:
preds_from_atac = {}
preds_from_rna = {}
true_atac = {}
for i in range(10):
    preds_from_atac[i] = adata_preds_from_atac.obsm[f"predicted_atac_{i}"]
    true_atac[i] = adata_target_atac.obsm["X_peakVI"]
    preds_from_rna[i] = adata_preds_from_rna.obsm[f"predicted_atac_{i}"][:,10:]
    


In [4]:
preds_identity = {0: adata_preds_from_atac.obsm["X_peakVI"]}

In [5]:
metrics_encoded_from_atac = jax.tree_util.tree_map(compute_metrics, preds_from_atac, true_atac)

2024-06-28 17:14:37.339472: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [6]:
compute_mean_metrics(metrics_encoded_from_atac)

{'r_squared': 0.9905986411804077,
 'sinkhorn_div_1': 2.2347991466522217,
 'sinkhorn_div_10': 0.47048587799072267,
 'sinkhorn_div_100': 0.20024528503417968,
 'e_distance': 0.3467470892186476,
 'mmd': 0.002353695882387304}

In [7]:
metrics_encoded_from_rna = jax.tree_util.tree_map(compute_metrics, preds_from_rna, true_atac)

In [8]:
compute_mean_metrics(metrics_encoded_from_rna)

{'r_squared': 0.9942269174178907,
 'sinkhorn_div_1': 2.9219343185424806,
 'sinkhorn_div_10': 0.6000816345214843,
 'sinkhorn_div_100': 0.15695571899414062,
 'e_distance': 0.1862438339751101,
 'mmd': 0.0026335724627036443}

In [9]:
metrics_identity = jax.tree_util.tree_map(compute_metrics, preds_identity, {0: true_atac[0]})

In [10]:
compute_mean_metrics(metrics_identity)

{'r_squared': 0.9172198370643104,
 'sinkhorn_div_1': 4.872247695922852,
 'sinkhorn_div_10': 3.1665306091308594,
 'sinkhorn_div_100': 1.8755989074707031,
 'e_distance': 3.2671523303988153,
 'mmd': 0.013792068571395552}

In [11]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/results/identity"

In [13]:
import os
import pandas as pd
pd.DataFrame.from_dict(metrics_encoded_from_rna).to_csv("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/results/genot_quad/metrics_encoded.csv")
pd.DataFrame.from_dict(metrics_encoded_from_atac).to_csv("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/results/genot_lin/metrics_encoded.csv")
pd.DataFrame.from_dict(metrics_identity).to_csv("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/results/identity/metrics_encoded.csv")
